In [1]:
%load_ext autoreload
%autoreload 2
%cd '/home/hew/python/genhance/ACE2'
%ls

/home/hew/python/genhance/ACE2
'Analyze Controlled Generators.ipynb'
'Analyze FoldX Results.ipynb'
 backup/
 generate_sequences_congen.py
 generation_scripts/
'Prepare Sequences for FoldX Cooptim.ipynb'
'Prepare Sequences for FoldX.ipynb'
 train_controlled_generator_continue.py
 train_controlled_generator_cooptim.py
 train_controlled_generator.py
 training_scripts/


In [2]:
import os
import re
import torch
import shutil
import scipy
import typing
import random
import numpy as np
import pandas as pd
import torch.nn.functional as F
from tqdm import tqdm
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer
from transformers.optimization import AdamW, get_scheduler
from transformers_custom import MT5ForConditionalGenerationWithLatentSpace, MT5Config
from tensorboardX import SummaryWriter

# Debug

In [3]:
pretrained_dir = 'Rostlab/prot_t5_xl_uniref50'
cache_dir = '/home/hew/storage/storage/genhance/pretrained/'
src_json = '/home/hew/python/genhance/temp/config.json'
clone_dir = '/home/hew/storage/storage/genhance/pretrained/prot_t5_xl_uniref50'
# pretrained_dir = '/home/hew/storage/storage/genhance/ckpts/congen1/results/step_35000'

In [4]:
print('Loading model')
tokenizer = T5Tokenizer.from_pretrained(pretrained_dir)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('device', device)
tokenizer.add_special_tokens({"cls_token": "<cls>"})
assert tokenizer.cls_token == "<cls>"

Loading model


device cuda:0


In [5]:
%%time
model = MT5ForConditionalGenerationWithLatentSpace.from_pretrained(pretrained_dir)
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

==================== T5 Model T5ForConditionalGenerationWithLatentSpace Initialization start ====================
========== Initialize T5ForConditionalGenerationWithLatentSpace ==========
latent_space_type:  wae
wae_z_enc_type:  deterministic
separate_latent_enc:  False
separate_latent_dec:  False
mmd_method:  rf
sigma_mmd:  None
rf_dim_mmd:  None
dim_target_kl:  0.5
latent_size:  1024
latent_pooler:  cls
pool_enc_hidden_states_for_dec:  True
mask_non_target_z_vector:  False
separate_targetattr_head:  False
do_mi:  False
========== Initialize T5ForConditionalGenerationWithLatentSpace ==========
==================== T5 Model T5ForConditionalGenerationWithLatentSpace Initialization end ====================


Some weights of MT5ForConditionalGenerationWithLatentSpace were not initialized from the model checkpoint at Rostlab/prot_t5_xl_uniref50 and are newly initialized: ['vae_enc.weight', 'vae_dec.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CPU times: user 3min 17s, sys: 29.9 s, total: 3min 47s
Wall time: 3min 48s


Embedding(129, 1024)

In [6]:
model

MT5ForConditionalGenerationWithLatentSpace(
  (shared): Embedding(129, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(129, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
 

In [7]:
%%time
temp_dir = '/home/hew/temp/'
os.mkdir(temp_dir)
saved_weights_file = temp_dir + 'pytorch_model.bin'
torch.save(model.state_dict(), saved_weights_file)
shutil.copy(src_json, temp_dir)

CPU times: user 7.01 s, sys: 11.2 s, total: 18.2 s
Wall time: 18.2 s


'/home/hew/temp/config.json'

In [8]:
%%time
model_2 = MT5ForConditionalGenerationWithLatentSpace.from_pretrained(cache_dir)

==================== T5 Model T5ForConditionalGenerationWithLatentSpace Initialization start ====================
========== Initialize T5ForConditionalGenerationWithLatentSpace ==========
latent_space_type:  wae
wae_z_enc_type:  deterministic
separate_latent_enc:  False
separate_latent_dec:  False
mmd_method:  rf
sigma_mmd:  None
rf_dim_mmd:  None
dim_target_kl:  0.5
latent_size:  1024
latent_pooler:  cls
pool_enc_hidden_states_for_dec:  True
mask_non_target_z_vector:  False
separate_targetattr_head:  False
do_mi:  False
========== Initialize T5ForConditionalGenerationWithLatentSpace ==========
==================== T5 Model T5ForConditionalGenerationWithLatentSpace Initialization end ====================


Some weights of the model checkpoint at /home/hew/storage/storage/genhance/pretrained/ were not used when initializing MT5ForConditionalGenerationWithLatentSpace: ['encoder.block.6.layer.0.SelfAttention.q.weight', 'encoder.block.6.layer.0.SelfAttention.k.weight', 'encoder.block.6.layer.0.SelfAttention.v.weight', 'encoder.block.6.layer.0.SelfAttention.o.weight', 'encoder.block.6.layer.0.layer_norm.weight', 'encoder.block.6.layer.1.DenseReluDense.wi.weight', 'encoder.block.6.layer.1.DenseReluDense.wo.weight', 'encoder.block.6.layer.1.layer_norm.weight', 'encoder.block.7.layer.0.SelfAttention.q.weight', 'encoder.block.7.layer.0.SelfAttention.k.weight', 'encoder.block.7.layer.0.SelfAttention.v.weight', 'encoder.block.7.layer.0.SelfAttention.o.weight', 'encoder.block.7.layer.0.layer_norm.weight', 'encoder.block.7.layer.1.DenseReluDense.wi.weight', 'encoder.block.7.layer.1.DenseReluDense.wo.weight', 'encoder.block.7.layer.1.layer_norm.weight', 'encoder.block.8.layer.0.SelfAttention.q.weight'

CPU times: user 21.7 s, sys: 11 s, total: 32.7 s
Wall time: 25.1 s


In [9]:
model_2

MT5ForConditionalGenerationWithLatentSpace(
  (shared): Embedding(129, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(129, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=4096, bias=False)
              (k): Linear(in_features=1024, out_features=4096, bias=False)
              (v): Linear(in_features=1024, out_features=4096, bias=False)
              (o): Linear(in_features=4096, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=1024, out_features=16384, bias=False)
              (wo): Linear(in_features=16384, out_features=1024, bias=False)
 

In [10]:
cache_config = '/home/hew/storage/storage/genhance/pretrained/config.json'

In [11]:
t5config = MT5Config.from_pretrained(cache_config)
t5config.num_layers = 6
t5config.num_decoder_layers = 6
t5config

MT5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 16384,
  "d_kv": 128,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 32,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.4.0.dev0",
  "use_cache": true,
  "vocab_size": 129
}

# Training

In [13]:
seed = 42
data_dir = '/home/hew/storage/storage/genhance/data/'
pretrained_dir = '/home/hew/storage/storage/genhance/pretrained/'
cache_dir = '/home/hew/storage/storage/genhance/pretrained/'
src_json = '/home/hew/python/genhance/temp/config.json'
output_dir = '/home/hew/storage/storage/genhance/ckpts/congen2/results/'
logging_dir = '/home/hew/python/genhance/tensorboard/congen2'
train_split_name = "train"
eval_split_name = "valid"
per_device_train_batch_size = 32
per_device_eval_batch_size = 128

save_total_limit = 2
lr = 1e-04
num_train_epochs = 30
train_ratio = 0.9
lambda_contrastive = 1.0
latent_pooler = 'cls'
pool_enc_hidden_states_for_dec = True
latent_space_type = 'wae'
beta_start_step = 200000
latent_size = 1024
wae_z_enc_type = 'deterministic'
no_separate_latent_enc = True
no_separate_latent_dec = True
lambda_contrastive_cyc = 1.0
contrastive_cyc_start_step = 200000
eval_steps = 250
save_steps = 1000
logging_steps = 50

mask_non_target_z_vector = False
separate_targetattr_head = False
z_tar_vector_dim = 1
do_mi = False
separate_latent_enc = False
separate_latent_dec = False
dim_target_kl = 0.5
mmd_method = 'rf'
sigma_mmd = None
rf_dim_mmd = None

warmup_steps = 500
num_warmup_steps = 0
weight_decay = 0.01
beta = 1.0
beta_ratio_increase = 0.25
beta_ratio_zero = 0.25
use_beta_schedule = False
lambda_logvar_L1 = 0.0
lambda_logvar_KL = 0.0
# lambda_contrastive_perturb_cyc = 0.0
# contrastive_perturb_cyc_start_step = -1
lambda_contrastive_perturb_cyc = 1.0
contrastive_perturb_cyc_start_step = 1
pc_perturb_type = 'std'
pc_perturb = -0.25

property = 'ddG'

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [5]:
class PKLDFDatasetForGen(Dataset):
    def __init__(
            self,
            data_file: typing.Union[str, Path],
            in_memory: bool = False,
            split: str = 'train',
            train_ratio: float = 1,
            data_subset='full'
            ):

        data_file = Path(data_file)
        if not data_file.exists():
            raise FileNotFoundError(data_file)

        df = pd.read_pickle(data_file)

        if train_ratio != 1:
            shuffled_df = df.sort_index()
            train_num_samples = int(len(shuffled_df)*train_ratio)
            if split == 'train':
                final_df = shuffled_df.iloc[:train_num_samples]
            elif split == 'valid':
                final_df = shuffled_df.iloc[train_num_samples:]
            else:
                final_df = df
        else:
            final_df = df

        # split into subset if not full training set
        if data_subset != 'full':
            ddG_sorted_final_df = final_df.sort_values(by='ddG', ascending=True)
            train_subset_num_samples = int(data_subset*len(ddG_sorted_final_df))
            final_df = ddG_sorted_final_df.iloc[:train_subset_num_samples]

        print("split: ", split)
        print("data_file: ", data_file)
        print("len(final_df): ", len(final_df))

        self.df = final_df
        num_examples = len(final_df)
        self._num_examples = num_examples

        if in_memory:
            cache = [None]*num_examples
            self._cache = cache

        self._in_memory = in_memory

    def __len__(self) -> int:
        return self._num_examples

    def __getitem__(self, index: int):
        if not 0 <= index < self._num_examples:
            raise IndexError(index)

        if self._in_memory and self._cache[index] is not None:
            item = self._cache[index]
        else:
            row = self.df.iloc[index]
            item = {}
            item['ddG'] = row['ddG']
            item['solubility'] = row['solubility']
            item['input_ids'] = row['MT_seq']
            item['labels'] = row['MT_seq']
            item['id'] = str(index)
            if self._in_memory:
                self._cache[index] = item

        return item


class CustomStabilityDatasetForGenLatentSpace(Dataset):
    def __init__(
            self,
            data_path: typing.Union[str, Path],
            split: str,
            tokenizer,
            in_memory: bool = False,
            train_ratio: float = 1,
            ):

        self.tokenizer = tokenizer

        if split == 'valid':
            file_prefix = 'train'
        else:
            file_prefix = split

        data_path = Path(data_path)
        data_file = f'{file_prefix}_tophalf_ddG_solubility.pkl'  # tophalf
        self.data = PKLDFDatasetForGen(data_path/data_file, in_memory, split, train_ratio, data_subset='full')

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int):
        item = self.data[index]
        input_ids = item['input_ids']
        labels = item['labels']
        if property == 'ddG':
            ddG = item['ddG']
            solubility = None
        elif property == 'solubility':
            ddG = None
            solubility = item['solubility']
        elif property == 'ddG_solubility':
            ddG = item['ddG']
            solubility = item['solubility']
        return input_ids, labels, ddG, solubility

    def collate_fn(self, batch: typing.List[typing.Tuple[typing.Any, ...]]) -> typing.Dict[str, torch.Tensor]:
        input_ids, labels, ddG, solubility = tuple(zip(*batch))
        prefix = "<cls> "
        # length = <cls> + 83 + </s> = 85
        input_ids = [prefix + " ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in input_ids]
        labels = [prefix + " ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in labels]
        input_ids = self.tokenizer.batch_encode_plus(input_ids, add_special_tokens=True, padding="longest", return_tensors='pt')[
            'input_ids']
        labels = self.tokenizer.batch_encode_plus(labels, add_special_tokens=True, padding="longest", return_tensors='pt')['input_ids']
        ddG = torch.Tensor(ddG) if None not in ddG else None
        solubility = torch.Tensor(solubility) if None not in solubility else None
        return {
            'input_ids' : input_ids,
            'labels'    : labels,
            'ddG'       : ddG,
            'solubility': solubility
            }


def spearmanr(target, prediction):
    target_array = np.asarray(target)
    prediction_array = np.asarray(prediction)
    return scipy.stats.spearmanr(target_array, prediction_array).correlation


def evaluate(model, eval_loader, do_mi=False, do_spearmanr=True, latent_space_type='wae', return_pred=False):
    eval_contrastive_loss_total = 0
    eval_lm_loss_total = 0
    if do_mi:
        eval_mi_head_loss_total = 0
    if latent_space_type in ['vae', 'wae']:
        eval_z_regu_loss_total = 0
    model.eval()
    num_eval_batch = 0

    ddG_preds = []
    ddG_targs = []
    solubility_preds = []
    solubility_targs = []

    with torch.no_grad():
        for step, batch in enumerate(tqdm(eval_loader)):
            input_ids = batch['input_ids'].to(model.device)
            labels = batch['labels'].to(model.device)
            ddG_targets = batch['ddG'].to(model.device) if 'ddG' in batch else None
            solubility_targets = batch['solubility'].to(model.device) if 'solubility' in batch else None
            contrast_targets = [ddG_targets, solubility_targets]

            if do_mi:
                model_outputs = model(input_ids, labels=labels, contrast_targets=contrast_targets)
                outputs, contrastive_loss, contrastive_value, mi_head_loss = model_outputs[0], model_outputs[1], model_outputs[2], \
                    model_outputs[3]
                eval_mi_head_loss_total = eval_mi_head_loss_total + mi_head_loss
            else:
                model_outputs = model(input_ids, labels=labels, contrast_targets=contrast_targets)
                outputs, contrastive_loss, contrastive_value = model_outputs[0], model_outputs[1], model_outputs[2]

            if latent_space_type in ['vae', 'wae']:
                z_regu_output = model_outputs[-1]
                if type(z_regu_output) is dict:
                    z_regu_loss = z_regu_output['z_regu_loss']
                else:
                    z_regu_loss = z_regu_output

            # contrastive_value: [batch_size, 1] or [batch_size, 2], ddG_targets: [batch_size]
            if property == 'ddG':
                for pred, target in zip(contrastive_value.squeeze().cpu().numpy(), ddG_targets.cpu().numpy()):
                    ddG_targs.append(target)
                    ddG_preds.append(pred)
            elif property == 'solubility':
                for pred, target in zip(contrastive_value.squeeze().cpu().numpy(), ddG_targets.cpu().numpy()):
                    solubility_targs.append(target)
                    solubility_preds.append(pred)
            elif property == 'ddG_solubility':
                for pred, target in zip(contrastive_value[:, 0].squeeze().cpu().numpy(), ddG_targets.cpu().numpy()):
                    ddG_targs.append(target)
                    ddG_preds.append(pred)
                for pred, target in zip(contrastive_value[:, 1].squeeze().cpu().numpy(), ddG_targets.cpu().numpy()):
                    solubility_targs.append(target)
                    solubility_preds.append(pred)

            lm_loss = outputs.loss

            eval_contrastive_loss_total = eval_contrastive_loss_total + contrastive_loss
            eval_lm_loss_total = eval_lm_loss_total + lm_loss

            if latent_space_type in ['vae', 'wae']:
                eval_z_regu_loss_total = eval_z_regu_loss_total + z_regu_loss

            num_eval_batch += 1

    eval_lm_loss = eval_lm_loss_total/num_eval_batch
    eval_contrastive_loss = eval_contrastive_loss_total/num_eval_batch
    eval_output = {
        "lm_loss"         : eval_lm_loss,
        "contrastive_loss": eval_contrastive_loss,
        }

    if do_mi:
        eval_mi_head_loss_total = eval_mi_head_loss_total/num_eval_batch
        eval_output['mi_head_loss'] = eval_mi_head_loss_total

    if latent_space_type in ['vae', 'wae']:
        eval_z_regu_loss_total = eval_z_regu_loss_total/num_eval_batch
        eval_output['z_regu_loss'] = eval_z_regu_loss_total

    if do_spearmanr:
        ddG_spearman = 0
        solubility_spearman = 0
        if len(ddG_targs) > 0 and len(ddG_preds) > 0:
            ddG_spearman = spearmanr(ddG_targs, ddG_preds)
        if len(solubility_targs) > 0 and len(solubility_preds) > 0:
            solubility_spearman = spearmanr(solubility_targs, solubility_preds)

        print()
        print("========== evaluation finished ==========")
        print("ddG_spearman: ", ddG_spearman)
        print("solubility_spearman: ", solubility_spearman)
        print("========== evaluation finished ==========")
        eval_output['ddG_spearman'] = ddG_spearman
        eval_output['solubility_spearman'] = solubility_spearman

    if return_pred:
        eval_output['ddG_preds'] = ddG_preds
        eval_output['ddG_targs'] = ddG_targs
        eval_output['solubility_preds'] = solubility_preds
        eval_output['solubility_targs'] = solubility_targs

    return eval_output


def frange_cycle_zero_linear(n_iter, start=0.0, stop=1.0, n_cycle=4, ratio_increase=0.5, ratio_zero=0.3):
    L = np.ones(n_iter)*stop
    period = n_iter/n_cycle
    step = (stop - start)/(period*ratio_increase)  # linear schedule

    for c in range(n_cycle):
        v, i = start, 0
        while v <= stop and (int(i + c*period) < n_iter):
            if i < period*ratio_zero:
                L[int(i + c*period)] = start
            else:
                L[int(i + c*period)] = v
                v += step
            i += 1
    return L


def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print("mkdir: ", path)

In [6]:
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)

output_dir = Path(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)

latent_space_type = latent_space_type
wae_z_enc_type = wae_z_enc_type
latent_space_args = {
    'latent_pooler'                 : latent_pooler,
    'pool_enc_hidden_states_for_dec': pool_enc_hidden_states_for_dec,
    'mask_non_target_z_vector'      : mask_non_target_z_vector,
    'separate_targetattr_head'      : separate_targetattr_head,
    'z_tar_vector_dim'              : z_tar_vector_dim,
    'do_mi'                         : do_mi,
    'latent_space_type'             : latent_space_type,
    'separate_latent_enc'           : separate_latent_enc,
    'separate_latent_dec'           : separate_latent_dec,
    'wae_z_enc_type'                : wae_z_enc_type,
    'latent_size'                   : latent_size,
    'dim_target_kl'                 : dim_target_kl,
    'mmd_method'                    : mmd_method,
    'sigma_mmd'                     : sigma_mmd,
    'rf_dim_mmd'                    : rf_dim_mmd,
    }

print('-'*100)
print("latent_space_args: ", latent_space_args)
print('-'*100)

----------------------------------------------------------------------------------------------------
latent_space_args:  {'latent_pooler': 'cls', 'pool_enc_hidden_states_for_dec': True, 'mask_non_target_z_vector': False, 'separate_targetattr_head': False, 'z_tar_vector_dim': 1, 'do_mi': False, 'latent_space_type': 'wae', 'separate_latent_enc': False, 'separate_latent_dec': False, 'wae_z_enc_type': 'deterministic', 'latent_size': 1024, 'dim_target_kl': 0.5, 'mmd_method': 'rf', 'sigma_mmd': None, 'rf_dim_mmd': None}
----------------------------------------------------------------------------------------------------


In [7]:
# load dataset
tokenizer = T5Tokenizer.from_pretrained('Rostlab/prot_t5_xl_uniref50', cache_dir=cache_dir)
tokenizer.add_special_tokens({"cls_token": "<cls>"})
assert tokenizer.cls_token == "<cls>"

train_dataset = CustomStabilityDatasetForGenLatentSpace(data_dir, train_split_name, train_ratio=train_ratio, tokenizer=tokenizer)
eval_dataset = CustomStabilityDatasetForGenLatentSpace(data_dir, eval_split_name, train_ratio=train_ratio, tokenizer=tokenizer)
num_training_steps = num_train_epochs*len(train_dataset)//per_device_train_batch_size

# Train data set-up
train_loader = DataLoader(train_dataset, batch_size=per_device_train_batch_size, shuffle=True,
                          num_workers=0, collate_fn=train_dataset.collate_fn)

# Eval data set-up
eval_loader = DataLoader(eval_dataset, batch_size=per_device_eval_batch_size, shuffle=False,
                         num_workers=0, collate_fn=train_dataset.collate_fn)

# load model (time consuming)
device = torch.device('cuda:0')
t5config = MT5Config.from_pretrained(pretrained_dir, cache_dir=cache_dir)
print('-'*100)
print('t5config', t5config)
print('-'*100)

split:  train
data_file:  /home/hew/storage/storage/genhance/data/train_tophalf_ddG_solubility.pkl
len(final_df):  112500
split:  valid
data_file:  /home/hew/storage/storage/genhance/data/train_tophalf_ddG_solubility.pkl
len(final_df):  12500
----------------------------------------------------------------------------------------------------
t5config MT5Config {
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 16384,
  "d_kv": 128,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 32,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "transformers_version": "4.4.0.dev0",
  "use_cache": true,
  "voc

In [8]:
%%time
model = MT5ForConditionalGenerationWithLatentSpace.from_pretrained(pretrained_dir, cache_dir=cache_dir, **latent_space_args)

print('='*50, 'parallelize', '='*50)
model.parallelize()
model.resize_token_embeddings(len(tokenizer))

==================== T5 Model T5ForConditionalGenerationWithLatentSpace Initialization start ====================
========== Initialize T5ForConditionalGenerationWithLatentSpace ==========
latent_space_type:  wae
wae_z_enc_type:  deterministic
separate_latent_enc:  False
separate_latent_dec:  False
mmd_method:  rf
sigma_mmd:  None
rf_dim_mmd:  None
dim_target_kl:  0.5
latent_size:  1024
latent_pooler:  cls
pool_enc_hidden_states_for_dec:  True
mask_non_target_z_vector:  False
separate_targetattr_head:  False
do_mi:  False
========== Initialize T5ForConditionalGenerationWithLatentSpace ==========
==================== T5 Model T5ForConditionalGenerationWithLatentSpace Initialization end ====================


Some weights of the model checkpoint at /home/hew/storage/storage/genhance/pretrained/ were not used when initializing MT5ForConditionalGenerationWithLatentSpace: ['encoder.block.6.layer.0.SelfAttention.q.weight', 'encoder.block.6.layer.0.SelfAttention.k.weight', 'encoder.block.6.layer.0.SelfAttention.v.weight', 'encoder.block.6.layer.0.SelfAttention.o.weight', 'encoder.block.6.layer.0.layer_norm.weight', 'encoder.block.6.layer.1.DenseReluDense.wi.weight', 'encoder.block.6.layer.1.DenseReluDense.wo.weight', 'encoder.block.6.layer.1.layer_norm.weight', 'encoder.block.7.layer.0.SelfAttention.q.weight', 'encoder.block.7.layer.0.SelfAttention.k.weight', 'encoder.block.7.layer.0.SelfAttention.v.weight', 'encoder.block.7.layer.0.SelfAttention.o.weight', 'encoder.block.7.layer.0.layer_norm.weight', 'encoder.block.7.layer.1.DenseReluDense.wi.weight', 'encoder.block.7.layer.1.DenseReluDense.wo.weight', 'encoder.block.7.layer.1.layer_norm.weight', 'encoder.block.8.layer.0.SelfAttention.q.weight'

================================================== parallelize ==================================================
CPU times: user 40.8 s, sys: 14.1 s, total: 54.9 s
Wall time: 38.4 s


Embedding(129, 1024)

In [9]:
model.device

device(type='cuda', index=0)

In [16]:
logging_dir = Path(logging_dir)
logging_dir.mkdir(parents=True, exist_ok=True)
tb_writer = SummaryWriter(logging_dir)

PermissionError: [Errno 13] Permission denied: '/home/hew/python/genhance/tensorboard/congen2/events.out.tfevents.1681060698.wenjia-genhance-spot-4'

In [10]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {
        'params'      : [p for n, p in model.named_parameters() if ('mi_head' not in n and not any(nd in n for nd in no_decay))],
        'weight_decay': 0.01
        },
    {
        'params'      : [p for n, p in model.named_parameters() if ('mi_head' not in n and any(nd in n for nd in no_decay))],
        'weight_decay': 0.0
        }
    ]
optimizer = AdamW(optimizer_grouped_parameters, lr=lr)

# lr scheduling
lr_scheduler = get_scheduler(
        'linear',
        optimizer,
        num_warmup_steps=num_warmup_steps,
        num_training_steps=num_training_steps,
        )

/home/hew/miniconda3/envs/genhance/lib/python3.8/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [11]:
global_step = 0
n_iter = int(num_train_epochs*len(train_loader))
print("len_train_loader: ", len(train_loader))
print("num_train_epochs: ", num_train_epochs)
print("n_iter: ", n_iter)

beta_t_list = frange_cycle_zero_linear(n_iter, start=0.0, stop=beta, n_cycle=10, ratio_increase=beta_ratio_increase,
                                       ratio_zero=beta_ratio_zero)
if beta_start_step > 0:
    beta_t_list[:beta_start_step] = 0

start_epoch = 1

len_train_loader:  3516
num_train_epochs:  30
n_iter:  105480


In [18]:
model.train()
for epoch in tqdm(range(start_epoch, num_train_epochs + 1)):
    print('start_epoch:', start_epoch, 'current epoch:', epoch, 'global_step:', global_step)
    for step, batch in enumerate(tqdm(train_loader)):
        input_ids = batch['input_ids'].to(model.device)
        labels = batch['labels'].to(model.device)
        ddG_targets = batch['ddG'].to(model.device) if batch['ddG'] is not None else None
        solubility_targets = batch['solubility'].to(model.device) if batch['solubility'] is not None else None
        contrast_targets = [ddG_targets, solubility_targets]
        model.zero_grad()

        model_outputs = model(input_ids, labels=labels, contrast_targets=contrast_targets)
        outputs, contrastive_loss, contrastive_value = model_outputs[0], model_outputs[1], model_outputs[2]
        lm_loss = outputs.loss

        '''reconstruction loss'''
        total_loss = lm_loss

        '''contrastive loss'''
        if lambda_contrastive > 0:
            if total_loss.device != contrastive_loss.device:
                contrastive_loss = contrastive_loss.to(total_loss.device)
            total_loss = total_loss + lambda_contrastive*contrastive_loss

        if latent_space_type in ['vae', 'wae']:
            z_regu_output = model_outputs[-1]
            if type(z_regu_output) is dict:
                z_regu_loss = z_regu_output['z_regu_loss']
            else:
                z_regu_loss = z_regu_output

            if use_beta_schedule and global_step < len(beta_t_list):
                '''no use here, use_beta_schedule == False'''
                beta_z_regu = beta_t_list[global_step]
            else:
                if beta_start_step > 0 and global_step < beta_start_step:
                    beta_z_regu = 0
                else:
                    beta_z_regu = beta  # constant value
            if beta_z_regu > 0:
                if total_loss.device != z_regu_loss.device:
                    z_regu_loss = z_regu_loss.to(total_loss.device)

                '''smoothness loss'''
                total_loss = total_loss + beta_z_regu*z_regu_loss

            if wae_z_enc_type != 'deterministic':
                '''no use here, wae_z_enc_type == deterministic'''
                z_logvar_L1 = z_regu_output['z_logvar_L1']
                if lambda_logvar_L1 > 0 and beta_z_regu > 0:
                    if total_loss.device != z_logvar_L1.device:
                        z_logvar_L1 = z_logvar_L1.to(total_loss.device)

                    # prevent z_logvar from being too large
                    total_loss = total_loss + beta_z_regu*lambda_logvar_L1*z_logvar_L1

                z_logvar_KL_penalty = z_regu_output['z_logvar_KL_penalty']
                if lambda_logvar_KL > 0 and beta_z_regu > 0:
                    if total_loss.device != z_logvar_KL_penalty.device:
                        z_logvar_KL_penalty = z_logvar_KL_penalty.to(total_loss.device)

                    # prevent z_logvar from diminishing
                    total_loss = total_loss + beta_z_regu*lambda_logvar_KL*z_logvar_KL_penalty

        # perturb cycle consistency loss
        '''no use here, lambda_contrastive_perturb_cyc == 0.0'''
        if lambda_contrastive_perturb_cyc > 0 and global_step > contrastive_perturb_cyc_start_step:
            if pc_perturb_type == 'std':
                contrastive_value_std = torch.std(contrastive_value)
                pc_perturb = pc_perturb*contrastive_value_std.item()
            elif pc_perturb_type == 'static':
                pc_perturb = pc_perturb

            gen_output = model.generate(input_ids, max_length=input_ids.shape[-1] + 1, return_dict_in_generate=True, output_scores=True,
                                        z_tar_edit_before_dec=pc_perturb)  # change z_tar_edit_before_dec

            # print('gen_output.scores', len(gen_output.scores), gen_output.scores[0].shape, gen_output.scores[1].shape)
            gen_logits = torch.stack(gen_output.scores, dim=1)
            # print('gen_logits', gen_logits.shape, gen_logits[0])
            pc_gen_value_pred = model(inputs_logits=gen_logits, return_only_value_pred=True)
            # print('pc_gen_value_pred', pc_gen_value_pred.shape)
            if len(ddG_targets.shape) != 2:
                ddG_targets = torch.unsqueeze(ddG_targets, dim=-1)
            pc_gen_ddG_targets = ddG_targets + pc_perturb
            pc_ddG_labels = torch.sign(ddG_targets - pc_gen_ddG_targets)*0.5 + 0.5
            contrastive_preds = F.logsigmoid(contrastive_value - pc_gen_value_pred)
            inverse_preds = F.logsigmoid(-1*(contrastive_value - pc_gen_value_pred))
            if pc_ddG_labels.device != contrastive_preds.device:
                pc_ddG_labels = pc_ddG_labels.to(contrastive_preds.device)
            pc_losses = -pc_ddG_labels*contrastive_preds - (1 - pc_ddG_labels)*inverse_preds
            contrastive_perturb_cyc_loss = pc_losses.mean()

            if total_loss.device != contrastive_perturb_cyc_loss.device:
                contrastive_perturb_cyc_loss = contrastive_perturb_cyc_loss.to(total_loss.device)
            total_loss = total_loss + lambda_contrastive_perturb_cyc*contrastive_perturb_cyc_loss

        if lambda_contrastive_cyc > 0 and global_step > contrastive_cyc_start_step:
            model_outputs_2nd_forward = model(inputs_logits=outputs.logits, labels=labels, contrast_targets=contrast_targets)
            contrastive_cyc_loss = model_outputs_2nd_forward[1]

            if total_loss.device != contrastive_cyc_loss.device:
                contrastive_cyc_loss = contrastive_cyc_loss.to(total_loss.device)

            '''cycle consistency loss'''
            total_loss = total_loss + lambda_contrastive_cyc*contrastive_cyc_loss

        total_loss.backward()
        optimizer.step()
        lr_scheduler.step()
        global_step += 1

        if global_step%logging_steps == 0:
            tb_writer.add_scalar("TRAIN/lr", lr_scheduler.get_last_lr()[0], global_step)
            tb_writer.add_scalar("TRAIN/contrastive_loss", contrastive_loss, global_step)
            tb_writer.add_scalar("TRAIN/lm_loss", lm_loss, global_step)
            if latent_space_type in ['vae', 'wae']:
                tb_writer.add_scalar("TRAIN/z_regu_loss", z_regu_loss, global_step)
                tb_writer.add_scalar("TRAIN/beta_z_regu", beta_z_regu, global_step)
                if wae_z_enc_type != 'deterministic':
                    tb_writer.add_scalar("TRAIN/z_logvar_L1", z_logvar_L1, global_step)
                    tb_writer.add_scalar("TRAIN/z_logvar_KL_penalty", z_logvar_KL_penalty, global_step)

            if lambda_contrastive_cyc > 0 and global_step > contrastive_cyc_start_step:
                tb_writer.add_scalar("TRAIN/contrastive_cyc_loss", contrastive_cyc_loss, global_step)

            if lambda_contrastive_perturb_cyc > 0 and global_step > contrastive_perturb_cyc_start_step:
                tb_writer.add_scalar("TRAIN/contrastive_perturb_cyc_loss", contrastive_perturb_cyc_loss, global_step)

        if global_step%eval_steps == 0:
            model.eval()
            eval_output = evaluate(model, eval_loader, do_mi=do_mi, latent_space_type=latent_space_type)
            eval_lm_loss, eval_contrastive_loss, eval_ddG_spearman, eval_solubility_spearman = eval_output['lm_loss'], eval_output[
                'contrastive_loss'], eval_output['ddG_spearman'], eval_output['solubility_spearman'],
            tb_writer.add_scalar("EVAL/lm_loss", eval_lm_loss, global_step)
            tb_writer.add_scalar("EVAL/contrastive_loss", eval_contrastive_loss, global_step)
            tb_writer.add_scalar("EVAL/ddG_spearmanr", eval_ddG_spearman, global_step)
            tb_writer.add_scalar("EVAL/solubility_spearmanr", eval_solubility_spearman, global_step)
            if do_mi:
                eval_mi_head_loss = eval_output['mi_head_loss']
                tb_writer.add_scalar("EVAL/mi_head_loss", eval_mi_head_loss, global_step)
            if latent_space_type in ['vae', 'wae']:
                eval_z_regu_loss = eval_output['z_regu_loss']
                tb_writer.add_scalar("EVAL/z_regu_loss", eval_z_regu_loss, global_step)
            model.train()

        if global_step%save_steps == 0:
            print()
            print('========== save step checkpoint ==========')
            weights_name = "pytorch_model.bin"
            ckpt_saved_dir = os.path.join(output_dir, f'step_{global_step}')
            mkdir(ckpt_saved_dir)
            saved_weights_file = os.path.join(ckpt_saved_dir, weights_name)
            torch.save(model.state_dict(), saved_weights_file)
            shutil.copy(src_json, ckpt_saved_dir)
            # torch.save(args, ckpt_saved_dir + "/training_bin")
            torch.save(optimizer.state_dict(), ckpt_saved_dir + "/optimizer.pt")
            torch.save(lr_scheduler.state_dict(), ckpt_saved_dir + "/scheduler.pt")
            ckpt_info = {'epoch': epoch, 'global_step': global_step}
            torch.save(ckpt_info, ckpt_saved_dir + "/ckpt_info.pt")

  0%|          | 0/30 [00:00<?, ?it/s]

start_epoch: 1 current epoch: 1 global_step: 17



  0%|          | 1/3516 [00:02<2:02:52,  2.10s/it]

gen_output.scores 85 torch.Size([32, 129]) torch.Size([32, 129])
gen_logits torch.Size([32, 85, 129]) tensor([[-4.1270e+01, -1.4107e+01, -1.2900e+02,  ..., -1.2632e+02,
         -1.3733e+00,  2.2185e-02],
        [-1.5435e+02, -2.8106e+01, -1.9018e+02,  ..., -1.8501e+02,
         -1.5003e+01,  2.3761e-02],
        [-1.0998e+02, -2.6472e+01, -1.7997e+02,  ..., -1.7474e+02,
         -6.0681e+00,  2.2423e-02],
        ...,
        [-6.5989e+01,  6.7634e+00, -1.3235e+02,  ..., -1.2796e+02,
          4.1947e-01,  9.0156e-04],
        [-7.4185e+01,  1.8686e+01, -1.4876e+02,  ..., -1.4401e+02,
          2.2148e+00,  1.4316e-02],
        [-9.5738e+01, -3.2716e+01, -1.6270e+02,  ..., -1.5683e+02,
         -7.5621e+00, -2.2270e-02]], device='cuda:0')
pc_gen_value_pred torch.Size([32, 1])



  0%|          | 2/3516 [00:04<2:10:15,  2.22s/it]

gen_output.scores 85 torch.Size([32, 129]) torch.Size([32, 129])
gen_logits torch.Size([32, 85, 129]) tensor([[-5.7531e+01, -2.4754e+01, -1.3775e+02,  ..., -1.3362e+02,
         -7.9772e+00,  4.3027e-02],
        [-1.1806e+02, -4.2069e+01, -1.7722e+02,  ..., -1.7296e+02,
         -1.1798e+01,  1.1884e-02],
        [-1.1035e+02, -2.6837e+01, -1.6398e+02,  ..., -1.5989e+02,
         -1.6421e+01,  4.4236e-02],
        ...,
        [-1.0613e+02, -1.9419e+01, -1.8238e+02,  ..., -1.7763e+02,
         -2.1746e+01,  1.0557e-02],
        [-1.7954e+02, -2.1155e+01, -1.6548e+02,  ..., -1.6130e+02,
         -1.7622e+01,  3.5217e-02],
        [-1.6850e+02, -3.9683e+01, -1.8033e+02,  ..., -1.7795e+02,
         -2.6483e+01,  6.3323e-02]], device='cuda:0')
pc_gen_value_pred torch.Size([32, 1])


  0%|          | 0/30 [00:04<?, ?it/s]


KeyboardInterrupt: 